In [1]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Create engine: engine
engine = create_engine('sqlite:///country_club.db')

In [3]:
# Execute query and store records in DataFrame: df
df = pd.read_sql_query("Select * from Members", engine)

In [4]:
df.head(5)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


### Q10. Produce a list of facilities with a total revenue less than 1000.The output of facility name and total revenue, sorted by revenue.

In [5]:
query = """
SELECT 
  sub2.name AS facilityname, 
  sub2.totalrevenue AS totalrevenue 
FROM 
  (
    SELECT 
      sub1.facilityname AS name, 
      SUM(sub1.revenue) AS totalrevenue 
    FROM 
      (
        SELECT 
          b.bookid, 
          f.name AS facilityname, 
          CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) ELSE b.slots * f.membercost END AS Revenue 
        FROM 
          Bookings AS b 
          LEFT JOIN Members AS m ON m.memid = b.memid 
          LEFT JOIN Facilities AS f ON f.facid = b.facid
      ) AS sub1 
    GROUP BY 
      sub1.facilityname
  ) AS sub2 
GROUP BY 
  facilityname 
HAVING 
  totalrevenue < 1000 
ORDER BY 
  totalrevenue DESC;
"""
pd.read_sql_query(query, engine)

,facilityname,totalrevenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [6]:
query = """
SELECT 
  sub2.memberName AS membername, 
  sub2.recommenderfirstname || ', ' || sub2.recommendersurname AS recommendername 
FROM 
  (
    SELECT 
      sub1.memberName AS memberName, 
      sub1.recommenderId AS memberId, 
      m.firstname AS recommenderfirstname, 
      m.surname AS recommendersurname 
    FROM 
      (
        SELECT 
          m2.memid AS memberId, 
          m1.firstname || ', ' || m1.surname AS memberName, 
          m2.recommendedby AS recommenderId 
        FROM 
          Members AS m1 
          INNER JOIN Members AS m2 ON m1.memid = m2.memid 
        WHERE 
          (
            m2.recommendedby IS NOT NULL 
            OR m2.recommendedby <> ' ' 
            OR m2.recommendedby <> ''
          ) 
          AND m1.memid <> 0
      ) AS sub1 
      LEFT JOIN Members AS m ON sub1.recommenderId = m.memid 
    WHERE 
      m.memid <> 0
  ) AS sub2;
"""
pd.read_sql_query(query, engine)

,membername,recommendername
0,"Janice, Joplette","Darren, Smith"
1,"Gerald, Butters","Darren, Smith"
2,"Nancy, Dare","Janice, Joplette"
3,"Tim, Boothe","Tim, Rownam"
4,"Ponder, Stibbons","Burton, Tracy"
5,"Charles, Owen","Darren, Smith"
6,"David, Jones","Janice, Joplette"
7,"Anne, Baker","Ponder, Stibbons"
8,"Jack, Smith","Darren, Smith"
9,"Florence, Bader","Ponder, Stibbons"


### Q12: Find the facilities with their usage by member, but not guests 

In [7]:
query = """
SELECT 
  f.name AS facilityname, 
  SUM(b.slots) AS slot_usage 
FROM 
  Bookings AS b 
  LEFT JOIN Facilities AS f ON f.facid = b.facid 
  LEFT JOIN Members AS m ON m.memid = b.memid 
WHERE 
  b.memid <> 0 
GROUP BY 
  facilityname 
ORDER BY 
  slot_usage DESC;

"""
pd.read_sql_query(query, engine)

,facilityname,slot_usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


### Q13: Find the facilities usage by month, but not guests 

In [8]:
query = """
SELECT 
  sub.MONTH AS MONTH, 
  sub.facilityname AS facility, 
  SUM(sub.slotNumber) AS slotusage 
FROM 
  (
    SELECT 
      strftime('%m', starttime) AS MONTH, 
      f.name AS facilityname, 
      b.slots AS slotNumber 
    FROM 
      Bookings AS b 
      LEFT JOIN Facilities AS f ON f.facid = b.facid 
      LEFT JOIN Members AS m ON m.memid = b.memid 
    WHERE 
      b.memid <> 0
  ) sub 
GROUP BY 
  MONTH, 
  facility 
ORDER BY 
  MONTH, 
  slotusage DESC;
"""
pd.read_sql_query(query, engine)

,MONTH,facility,slotusage
0,07,Tennis Court 1,201
1,07,Massage Room 1,166
2,07,Badminton Court,165
3,07,Snooker Table,140
4,07,Tennis Court 2,123
5,07,Pool Table,110
6,07,Table Tennis,98
7,07,Squash Court,50
8,07,Massage Room 2,8
9,08,Badminton Court,414
